In [3]:
! aws s3 ls s3://olgabot-maca/10x/10X_P4_3/

2018-10-12 22:36:53  237638698 possorted_genome_bam_scaled=0_num_hashes=10000.sig
2018-10-12 22:36:25   84626832 possorted_genome_bam_scaled=1000_num_hashes=0.sig


In [6]:
oldplace = 's3://olgabot-maca/10x/10X_P4_3/possorted_genome_bam_scaled=0_num_hashes=10000.sig'
newplace = 's3://olgabot-maca/10x/10X_P4_3/scaled=0_num_hashes=10000/possorted_genome_bam.sig'

! aws s3 mv $oldplace $newplace

move: s3://olgabot-maca/10x/10X_P4_3/possorted_genome_bam_scaled=0_num_hashes=10000.sig to s3://olgabot-maca/10x/10X_P4_3/scaled=0_num_hashes=10000/possorted_genome_bam.sig


In [7]:
oldplace = 's3://olgabot-maca/10x/10X_P4_3/possorted_genome_bam_scaled=1000_num_hashes=0.sig'
newplace = 's3://olgabot-maca/10x/10X_P4_3/scaled=1000_num_hashes=0/possorted_genome_bam.sig'

! aws s3 mv $oldplace $newplace

move: s3://olgabot-maca/10x/10X_P4_3/possorted_genome_bam_scaled=1000_num_hashes=0.sig to s3://olgabot-maca/10x/10X_P4_3/scaled=1000_num_hashes=0/possorted_genome_bam.sig


In [8]:
! aws s3 ls s3://olgabot-maca/10x/10X_P4_3/

                           PRE scaled=0_num_hashes=10000/
                           PRE scaled=1000_num_hashes=0/


In [4]:
! aws s3 cp /mnt/data/10X_P7_8/possorted_genome_bam.sig s3://olgabot-maca/10x/10X_P7_8/scaled=1000_num_hashes=0/possorted_genome_bam.sig

upload: ../../../../mnt/data/10X_P7_8/possorted_genome_bam.sig to s3://olgabot-maca/10x/10X_P7_8/scaled=1000_num_hashes=0/possorted_genome_bam.sig


In [11]:
lung_signatures = 's3://olgabot-maca/10x/10X_P7_8/scaled=1000_num_hashes=0/'
bladder_signatures = 's3://olgabot-maca/10x/10X_P4_3/scaled=1000_num_hashes=0/'

In [28]:
2+5

7

In [30]:
from itertools import product
import os

import pandas as pd
pd.options.display.max_colwidth = 500

ksizes = 21, 27, 33, 51

sequence_to_compare = 'dna', 'protein'


data = list(product(ksizes, sequence_to_compare))

runs = pd.DataFrame(data, columns=['ksize', 'sequence_to_compare'])
runs['signatures'] = lung_signatures
runs['output'] = runs['signatures'].str.replace('10x', '10x/sourmash_compare').str.rstrip('/') \
    + "_" + runs.sequence_to_compare + '_ksize=' + runs.ksize.astype(str) + '.csv'
runs['id'] = runs.output.map(os.path.basename) 
runs = runs.set_index('id')
runs.head()

,ksize,sequence_to_compare,signatures,output
id,,,,
scaled=1000_num_hashes=0_dna_ksize=21.csv,21,dna,s3://olgabot-maca/10x/10X_P7_8/scaled=1000_num_hashes=0/,s3://olgabot-maca/10x/sourmash_compare/10X_P7_8/scaled=1000_num_hashes=0_dna_ksize=21.csv
scaled=1000_num_hashes=0_protein_ksize=21.csv,21,protein,s3://olgabot-maca/10x/10X_P7_8/scaled=1000_num_hashes=0/,s3://olgabot-maca/10x/sourmash_compare/10X_P7_8/scaled=1000_num_hashes=0_protein_ksize=21.csv
scaled=1000_num_hashes=0_dna_ksize=27.csv,27,dna,s3://olgabot-maca/10x/10X_P7_8/scaled=1000_num_hashes=0/,s3://olgabot-maca/10x/sourmash_compare/10X_P7_8/scaled=1000_num_hashes=0_dna_ksize=27.csv
scaled=1000_num_hashes=0_protein_ksize=27.csv,27,protein,s3://olgabot-maca/10x/10X_P7_8/scaled=1000_num_hashes=0/,s3://olgabot-maca/10x/sourmash_compare/10X_P7_8/scaled=1000_num_hashes=0_protein_ksize=27.csv
scaled=1000_num_hashes=0_dna_ksize=33.csv,33,dna,s3://olgabot-maca/10x/10X_P7_8/scaled=1000_num_hashes=0/,s3://olgabot-maca/10x/sourmash_compare/10X_P7_8/scaled=1000_num_hashes=0_dna_ksize=33.csv


In [31]:
# samples = pd.DataFrame({'ksizes': ksizes})
# samples['signatures'] = lung_signatures
# samples['ignore_abundance'] = True
# samples['sequence_to_compare'] = 'dna'

In [32]:
import json

lines = []

program = 'compare'
folder = f'../sourmash/10x_lung_{program}'
! mkdir $folder
            
samples = runs.copy()
samples['output'] = samples.output.str.replace('PROGRAM', program) + '.csv'
samples.to_csv(f'{folder}/samples.csv')

config = 	{
		"program": f"../../reflow/sourmash_{program}.rf",
		"runs_file": "samples.csv"
	}


with open(f'{folder}/config.json', 'w') as f:
    json.dump(config, f)
    
! ls -lha $folder

total 16K
drwxrwxr-x  2 ubuntu ubuntu 4.0K Oct 16 17:56 .
drwxrwxr-x 13 ubuntu ubuntu 4.0K Oct 16 17:55 ..
-rw-rw-r--  1 ubuntu ubuntu   75 Oct 16 17:56 config.json
-rw-rw-r--  1 ubuntu ubuntu 1.7K Oct 16 17:56 samples.csv
